![alt text](logo.png "Logo")

<center> <h1> Advanced Physics Lab 3711/3721 </h1> </center>
<center> <h1> Plasma Physics - Langmuir Probe </h1> </center>


## Notebook Overview

Meet Jupyter Notebook! The Jupyter Notebook is a web application that allows you to create and share documents that contain live code, equations, visualizations and explanatory text. Uses include: data cleaning and transformation, numerical simulation, statistical modeling, machine learning and much more. It is an open source, interactive data science and scientific computing platform across over 40 programming languages.


## Let's get to it! Experimental Setup

Below is a digital photograph of the current experimental setup is NC 3107A. If what you find in the lab look different from the image, please contact an instructor or the lab coordinator before proceding.

<b>WARNING:</b> We will be dealing with large potential differences (<b>HIGH-VOLTAGE!</b>) in this lab. Please use caution and be aware of what you are touching. Familiarize yourself with the equipment before turning anything on.

![alt text](overview.jpg "Experimental Setup")

We first want to configure the Digital Multimeters (DMM). We want to measure both DC Current, and DC Voltage. Convince yourself what each DMM is set-up to measure, then turn them on. <i><b>Note:</b> The default Current setting is for AC Current, not DC. Be sure to change the function by pressing the yellow function button. </i>

You should have something like this:

![alt text](dmm.jpg "DMM Configuration")

Now, it's time to create a plasma! <b>Before</b> turning on the <b>High-Voltage</b> power supply, make sure that both the black (coarse) and red (fine) knobs, as seen below, are turned completely counter-clockwise. After verifying this, flip the "POWER ON" switch up.

![alt text](high_voltage.jpg "High Voltage")

Now we will (slowly) increase the potential between the Cathode and Anode inside the vacuum tube. You should be monitoring three things while increasing the potential:

+ DMM Current Reading
+ DMM Voltage Reading
+ <b>THE PLASMA THAT YOU ARE MAKING!!!</b>

You should expect to see a current when the potential reaches about 90 V (and see the plasma!). <b>Slowly</b> increase the potential (using the black knob) until the curent reaches about 28 mA. Then switch to the red knob and dial it up until the current is 30 mA. <b>Do not exceed 30 mA.</b>

Your DMM's should give similar values as pictured below:

![alt text](stop_here.jpg "Stop Here")

Look at the vacuum tube! The plasma should be nice and uniform, and not flickering. Something like this:

![alt text](plasma.jpg "Plasma")

Now, we get to sweep the voltage on the Langmuir Probe in the tube and measure current on the Keithley 6487 Picoammeter/Voltage Source!

## Instrument Control and Data Acquisition

This particular notebook is written to communicate with the Keithley 6487 Picoammeter/Voltage Source. The programming language used in this Notebook is Python. The best way to use this section is a top-down design. Each cell has been designed to be executed one at a time, but many of the cells require the ones above it to have been executed. If you have not used a Jupyter notebook, please speak with Kris Bunker before running it for a quick overview of it's functionality.

The first thing we need to do is import some Python libraries to communicate with the Keithley, and to manipulate and display the acquired data. Execute the cell below by clicking on the cell and press "Shift+Enter" (alternatively you could click on the cell and then click the play button on the top banner). <i><b>Note:</b> On the left of each "code" cell, you will see "In [ ]:". After you execute a cell, this will change to "In [ # ]:", where # will be an integer that represents the order you executed the cells in a particular session (beginning with 1 and monotonically inscreasing). </i>

In [ ]:
import numpy as np # This is the fundamental package for scientific computing with Python.
import matplotlib.pyplot as plt # A Python plotting library.
import visa # This is the pyVISA library - it allows us to communicate with instruments (like the Keithley 6487!).
import time # This library allows us to keep track of time
from bokeh.plotting import figure, output_notebook, show # Plotting library

Now we will open communication with the Keithley. It is located at the "COM3" serial port. We need to also define how the device is configured in the RS-232 communications protocol. In general, you need to specify:

+ Baud Rate
+ Flow Control
+ Terminator Characters
+ Parity
+ Data Bits

For our case, the Keithley is set to all defaults except the Terminator. The settings for the the Keithley are:    

+ Baud Rate: 9600
+ Flow Control: None
+ Terminator Characters: Carriage Return
+ Parity: None
+ Data Bits: 8

Now, run the cell below.

In [ ]:
# Setting up a resource manager to communicate with the Keithley, using the imported pyVISA library.
resource_manager = visa.ResourceManager() 

# Opening the instrument - note we specify the read and write termination
keithley = resource_manager.open_resource("COM3", read_termination = '\r', write_termination = '\r') 

Next, we define our scanning limits and other variables. Some notes about these parameters:

+ v_1 - This is the starting voltage. -30 V is a good starting point. Don't go any lower.
+ v_2 - This is the ending voltage. Do not exceed 9 V.
+ v_d - This is the voltage step. 1 V is a good starting point. Decreasing to 0.1 V for subsequent data runs should produce good results.
+ t_d - This is our time between voltage steps. You can keep this at 1 second.

In [ ]:
v_1 = -30 # Starting Voltage
v_2 = 9  # Ending Voltage
v_d = 1 # Voltage Step
t_d = 1 # Time (in seconds) between voltage ramps

N_steps = (v_2 - v_1)/v_d # Number of steps
vs = np.arange(v_1, v_2 +1, v_d) # generate an array of Voltage values

Now let's communicate with the Keithley 6487 to setup for the desired data acquisition.

In [ ]:
keithley.write('*RST') # Return 6487 to defaults.
keithley.write('FORM:ELEM READ,VSO') # Setting up Keithley to send the Current and Voltage.
keithley.write('SYST:ZCH ON') # Enable zero check.
keithley.write('RANG 2e-3') # Select the 2mA range.
keithley.write('INIT') # Trigger reading to be used as zero correction.
keithley.write('SYST:ZCOR:ACQ') # Use last reading taken as zero correct value.
keithley.write('SYST:ZCOR ON') # Perform zero correction.
keithley.write('RANG:AUTO ON') # Enable auto current range.
keithley.write('SOUR:VOLT:RANG 30') # Select 30V source range.
keithley.write('SOUR:VOLT 0') # Set voltage source output to 0V.
keithley.write('SOUR:VOLT:ILIM 2.5e-3') # Set current limit to 2.5mA.
keithley.write('SYST:ZCH OFF') # Disable zero check.

Alright. Now we should have the Keithley in a state where we can sweep the voltages. I would encourage you to read through the cell below and try to interpret what we are doing. Okay - let's go!

In [ ]:
# This cell is designed to be run multiple times as to collect multiple data sets. So, we will delete the previous data
# if it exists. So, if you want to keep your data, save it in a cell below!
if 'data' in locals(): 
    del data

keithley.write('SOUR:VOLT:STAT ON') # Put voltage source in operate.

data = np.empty((N_steps, 2)) # Create an empty arraythat we will fill with data below

i = 0 # Creating an index variable for storing data

# Looping through the voltages we defined a few cells back
for v in vs:
            keithley.write('SOUR:VOLT ' + str(v)) # Telling the Keithley a voltage to set
            time.sleep(t_d) # Wait for the time that we specified
            a = keithley.ask('READ?') # Reading the Current and Voltage and storing it
            b = a.split(",") # Parsing the data sent from the Keithley - it sends us a 'string'
            data[i,1] = new_y = float(b[0]) # Converting the 'string' to a 'float'
            data[i,0] = float(b[1]) # Converting the 'string' to a 'float'
                      
            i = i + 1 # increase our index variable before running through the loop again
                    
keithley.write('SOUR:VOLT 0')        
keithley.write('SOUR:VOLT:STAT OFF')

### If you received an error after running the cell above, and the blue "VOLTAGE SOURCE OPERATE" light is illuminated on the Keithley, execute the cell below. Otherwise ignore.

In [ ]:
keithley.write('SOUR:VOLT 0')        
keithley.write('SOUR:VOLT:STAT OFF')

When you are completely finished with your data collection, please close out the data acquisition session by executing the cell below. You may not wish to do this the first time you arrive to this point. Move on to the next section before determining if you are finished.

In [ ]:
keithley.close()

## Data Plotting

Now that we have acquired the data, let's have a look at the result! The cell below will create an interactive plot of the IV curve.

In [ ]:
# Plot the data

# Specifying that we want the plot to be displayed within the notebook
output_notebook(url=None, docname=None, session=None, name=None, resources=None, verbose=False, hide_banner=True)

# create a new plot with a title and axis labels
p = figure(title="Langmuir IV Curve", x_axis_label='Voltage (V)', y_axis_label='Current (A)')

# specify the data to plot and define the line type - here we will use a circle for the data points
p.circle(data[:,0], data[:,1], fill_color="white", size=8)

# show the results
show(p)

## Saving your data

Specify the filename for saving your data below. At this time the following cell WILL OVERWRITE any file that has the 
same name, so be sure to change the "filename" before executing this cell if this is not desired.

In [ ]:
np.savetxt('filename.txt', data,
                      fmt='%.18e',
                      delimiter=',',
                      newline='\n',
                      header='Voltage (V) Current (A)',
                      footer='',
                      comments='# ')